# Advanced Querying Mongo

Importing libraries and setting up connection

In [3]:
# Importing libraries
from pymongo import MongoClient


In [5]:
# Setting up connection
client = MongoClient('localhost:27017')
client

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)

In [7]:
# Getting dataframe
db = client.get_database('Ironhack')
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'Ironhack')

In [8]:
# Getting collection
companies = db.get_collection('companies')
companies

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'Ironhack'), 'companies')

### 1. All the companies whose name match 'Babelgum'. Retrieve only their `name` field.

In [9]:
# Checking 'companies' items
companies.find_one()

{'_id': ObjectId('52cdef7c4bab8bd675297d8a'),
 'name': 'Wetpaint',
 'permalink': 'abc2',
 'crunchbase_url': 'http://www.crunchbase.com/company/wetpaint',
 'homepage_url': 'http://wetpaint-inc.com',
 'blog_url': 'http://digitalquarters.net/',
 'blog_feed_url': 'http://digitalquarters.net/feed/',
 'twitter_username': 'BachelrWetpaint',
 'category_code': 'web',
 'number_of_employees': 47,
 'founded_year': 2005,
 'founded_month': 10,
 'founded_day': 17,
 'deadpooled_year': 1,
 'tag_list': 'wiki, seattle, elowitz, media-industry, media-platform, social-distribution-system',
 'alias_list': '',
 'email_address': 'info@wetpaint.com',
 'phone_number': '206.859.6300',
 'description': 'Technology Platform Company',
 'created_at': datetime.datetime(2007, 5, 25, 6, 51, 27),
 'updated_at': 'Sun Dec 08 07:15:44 UTC 2013',
 'overview': '<p>Wetpaint is a technology platform company that uses its proprietary state-of-the-art technology and expertise in social media to build and monetize audiences for di

In [13]:
filt = {'name': 'Babelgum'}
proj = {'_id': 0, 'name':1}

list(companies.find(filt, proj))

[{'name': 'Babelgum'}]

### 2. All the companies that have more than 5000 employees. Limit the search to 20 companies and sort them by **number of employees**.

In [19]:
filt = {'number_of_employees': {'$gt': 5000}}
proj = {'_id': 0, 'name': 1, 'number_of_employees': 1}

list(companies.find(filt, proj).sort('number_of_employees', -1).limit(20))

[{'name': 'Siemens', 'number_of_employees': 405000},
 {'name': 'IBM', 'number_of_employees': 388000},
 {'name': 'Toyota', 'number_of_employees': 320000},
 {'name': 'PayPal', 'number_of_employees': 300000},
 {'name': 'Nippon Telegraph and Telephone Corporation',
  'number_of_employees': 227000},
 {'name': 'Samsung Electronics', 'number_of_employees': 221726},
 {'name': 'Accenture', 'number_of_employees': 205000},
 {'name': 'Tata Consultancy Services', 'number_of_employees': 200300},
 {'name': 'Flextronics International', 'number_of_employees': 200000},
 {'name': 'Safeway', 'number_of_employees': 186000},
 {'name': 'Sony', 'number_of_employees': 180500},
 {'name': 'LG', 'number_of_employees': 177000},
 {'name': 'Ford', 'number_of_employees': 171000},
 {'name': 'Boeing', 'number_of_employees': 160000},
 {'name': 'Digital Equipment Corporation', 'number_of_employees': 140000},
 {'name': 'Nokia', 'number_of_employees': 125000},
 {'name': 'MItsubishi Electric', 'number_of_employees': 107000}

### 3. All the companies founded between 2000 and 2005, both years included. Retrieve only the `name` and `founded_year` fields.

In [24]:
# + Limited to 5 
proj = {"_id":0, "name":1, "founded_year":1}
filt = {'founded_year': {'$in': [2000, 2001, 2002, 2003, 2004, 2005]}}
        
list(companies.find(filt, proj).limit(5))

[{'name': 'Wetpaint', 'founded_year': 2005},
 {'name': 'Zoho', 'founded_year': 2005},
 {'name': 'Digg', 'founded_year': 2004},
 {'name': 'Facebook', 'founded_year': 2004},
 {'name': 'Omnidrive', 'founded_year': 2005}]

### 4. All the companies that had a Valuation Amount of more than 100.000.000 and have been founded before 2010. Retrieve only the `name` and `ipo` fields.

In [26]:
proj = {"_id": 0, "name": 1, "ipo":1}
filt = {"$and": [{"ipo.valuation_amount": {"$gt": 100_000_000}},{"founded_year": {"$lt":2010}}]}

list(companies.find(filt, proj).limit(5))

[{'name': 'Facebook',
  'ipo': {'valuation_amount': 104000000000,
   'valuation_currency_code': 'USD',
   'pub_year': 2012,
   'pub_month': 5,
   'pub_day': 18,
   'stock_symbol': 'NASDAQ:FB'}},
 {'name': 'Twitter',
  'ipo': {'valuation_amount': 18100000000,
   'valuation_currency_code': 'USD',
   'pub_year': 2013,
   'pub_month': 11,
   'pub_day': 7,
   'stock_symbol': 'NYSE:TWTR'}},
 {'name': 'Yelp',
  'ipo': {'valuation_amount': 1300000000,
   'valuation_currency_code': 'USD',
   'pub_year': 2012,
   'pub_month': 3,
   'pub_day': 2,
   'stock_symbol': 'NYSE:YELP'}},
 {'name': 'LinkedIn',
  'ipo': {'valuation_amount': 9310000000,
   'valuation_currency_code': 'USD',
   'pub_year': 2011,
   'pub_month': 7,
   'pub_day': 20,
   'stock_symbol': 'NYSE:LNKD'}},
 {'name': 'Amazon',
  'ipo': {'valuation_amount': 100000000000,
   'valuation_currency_code': 'USD',
   'pub_year': 1997,
   'pub_month': 5,
   'pub_day': None,
   'stock_symbol': 'NASDAQ:AMZN'}}]

### 5. All the companies that have less than 1000 employees and have been founded before 2005. Order them by the number of employees and limit the search to 10 companies.

In [32]:
proj = {"_id": 0, "name": 1, "number_of_employees": 1}
filt = {"$and": [{"number_of_employees": {"$lt": 1000}},{"founded_year": {"$lt":2005}}]}

list(companies.find(filt, proj).sort("number_of_employees",-1).limit(10))

[{'name': 'Infinera Corporation', 'number_of_employees': 974},
 {'name': 'NorthPoint Communications Group', 'number_of_employees': 948},
 {'name': '888 Holdings', 'number_of_employees': 931},
 {'name': 'Forrester Research', 'number_of_employees': 903},
 {'name': 'Webmetrics', 'number_of_employees': 900},
 {'name': 'SonicWALL', 'number_of_employees': 900},
 {'name': 'Cornerstone OnDemand', 'number_of_employees': 881},
 {'name': 'Buongiorno', 'number_of_employees': 800},
 {'name': 'Cvent', 'number_of_employees': 800},
 {'name': 'ZoomInfo', 'number_of_employees': 800}]

### 6. All the companies that don't include the `partners` field.

In [35]:
# Aquí obviamente he hecho algo mal porque no me da resultado 
proj = {"_id": 0, "name": 1, 'number_of_employees': 1}
filt = {"partners": {"$exists": False}}

list(companies.find(filt, proj))

[]

### 7. All the companies that have a null type of value on the `category_code` field.

In [37]:
proj = {"_id": 0, "name": 1}
filt = {"category_code": None}

list(companies.find(filt, proj).limit(5))

[{'name': 'Collective'},
 {'name': 'Snimmer'},
 {'name': 'KoolIM'},
 {'name': 'Level9 Media'},
 {'name': 'VidKing'}]

### 8. All the companies that have at least 100 employees but less than 1000. Retrieve only the `name` and `number of employees` fields.

In [39]:
proj = {"_id": 0, "name": 1, "number_of_employees": 1}
filt = {"$and": [{"number_of_employees": {"$gte": 100}}, {"number_of_employees": {"$lte": 1000}}]}

list(companies.find(filt, proj).limit(5))

[{'name': 'AdventNet', 'number_of_employees': 600},
 {'name': 'AddThis', 'number_of_employees': 120},
 {'name': 'OpenX', 'number_of_employees': 305},
 {'name': 'LifeLock', 'number_of_employees': 644},
 {'name': 'Jajah', 'number_of_employees': 110}]

### 9. Order all the companies by their IPO price in a descending order.

In [43]:
# ¿hay otro modo de sacar todos? si no uso filt me sale vacía
proj = {"_id": 0, "name": 1, "ipo.valuation_amount": 1}
filt = {"ipo.valuation_amount": {"$ne": None}}

list(companies.find(filt, proj).sort("ipo.valuation_amount", -1).limit(5))

[{'name': 'GREE', 'ipo': {'valuation_amount': 108960000000}},
 {'name': 'Facebook', 'ipo': {'valuation_amount': 104000000000}},
 {'name': 'Amazon', 'ipo': {'valuation_amount': 100000000000}},
 {'name': 'Twitter', 'ipo': {'valuation_amount': 18100000000}},
 {'name': 'Groupon', 'ipo': {'valuation_amount': 12800000000}}]

### 10. Retrieve the 10 companies with more employees, order by the `number of employees`

In [44]:
proj = {"_id": 0, "name": 1, "number_of_employees": 1}
filt = {"number_of_employees": {"$gt": 1}}

list(companies.find(filt, proj).sort("number_of_employees", -1).limit(10))

[{'name': 'Siemens', 'number_of_employees': 405000},
 {'name': 'IBM', 'number_of_employees': 388000},
 {'name': 'Toyota', 'number_of_employees': 320000},
 {'name': 'PayPal', 'number_of_employees': 300000},
 {'name': 'Nippon Telegraph and Telephone Corporation',
  'number_of_employees': 227000},
 {'name': 'Samsung Electronics', 'number_of_employees': 221726},
 {'name': 'Accenture', 'number_of_employees': 205000},
 {'name': 'Tata Consultancy Services', 'number_of_employees': 200300},
 {'name': 'Flextronics International', 'number_of_employees': 200000},
 {'name': 'Safeway', 'number_of_employees': 186000}]

### 11. All the companies founded on the second semester of the year. Limit your search to 1000 companies.

In [48]:
# Te limito a 10 en vez de 1000, que sino la lista será eterna en tu pantalla
proj = {"_id": 0, "name": 1}
filt = {"founded_month": {"$in" : [7, 8, 9, 10, 11, 12]}}

list(companies.find(filt, proj).limit(10))

[{'name': 'Wetpaint'},
 {'name': 'Zoho'},
 {'name': 'Digg'},
 {'name': 'Omnidrive'},
 {'name': 'eBay'},
 {'name': 'Joost'},
 {'name': 'Plaxo'},
 {'name': 'Powerset'},
 {'name': 'Kyte'},
 {'name': 'Thoof'}]

### 12. All the companies founded before 2000 that have an acquisition amount of more than 10.000.00

In [60]:
proj = {"_id": 0, "name": 1}
filt = {"$and": [{"founded_year": {"$lt": 2000}},{"acquisition.price_amount": {"$gt":10_000_000}}]}

list(companies.find(filt, proj).limit(5))

[{'name': 'Postini'},
 {'name': 'SideStep'},
 {'name': 'Recipezaar'},
 {'name': 'PayPal'},
 {'name': 'Snapfish'}]

### 13. All the companies that have been acquired after 2010, order by the acquisition amount, and retrieve only their `name` and `acquisition` field.

In [61]:
proj = {"_id": 0, "name": 1, "acquisition": 1}
filt = {"acquisition.acquired_year": {"$gt":2010}}

list(companies.find(filt, proj).sort("acquisition.price_amount", -1).limit(3))

[{'name': 'T-Mobile',
  'acquisition': {'price_amount': 39000000000,
   'price_currency_code': 'USD',
   'term_code': None,
   'source_url': 'http://techcrunch.com/2011/03/20/in-the-race-for-more-spectrum-att-is-acquiring-t-mobile-for-39-billion/',
   'source_description': 'In The Race For More Spectrum, AT&T Is Acquiring T-Mobile For $39 Billion',
   'acquired_year': 2011,
   'acquired_month': 3,
   'acquired_day': 20,
   'acquiring_company': {'name': 'AT&T', 'permalink': 'at-t'}}},
 {'name': 'Goodrich Corporation',
  'acquisition': {'price_amount': 18400000000,
   'price_currency_code': 'USD',
   'term_code': None,
   'source_url': 'http://www.masshightech.com/stories/2011/09/19/daily37-UTC-shells-out-184-billion-for-Goodrich.html',
   'source_description': 'UTC shells out $18.4 billion for Goodrich',
   'acquired_year': 2011,
   'acquired_month': 9,
   'acquired_day': 22,
   'acquiring_company': {'name': 'United Technologies',
    'permalink': 'united-technologies'}}},
 {'name': 'LS

### 14. Order the companies by their `founded year`, retrieving only their `name` and `founded year`.

In [62]:
proj = {"_id": 0, "name": 1, "founded_year": 1}
filt = {"founded_year": {"$ne": None}}

list(companies.find(filt, proj).sort("founded_year", -1).limit(5))

[{'name': 'Fixya', 'founded_year': 2013},
 {'name': 'iBazar', 'founded_year': 2013},
 {'name': 'Fluc', 'founded_year': 2013},
 {'name': 'Advaliant', 'founded_year': 2013},
 {'name': 'Wamba', 'founded_year': 2013}]

### 15. All the companies that have been founded on the first seven days of the month, including the seventh. Sort them by their `acquisition price` in a descending order. Limit the search to 10 documents.

In [63]:
proj = {"_id": 0, "name": 1, "acquisition.price_amount": 1}
filt = {"founded_day": {"$in": [1,2,3,4,5,6,7]}}

list(companies.find(filt, proj).sort("acquisition.price_amount", -1).limit(10))

[{'name': 'Netscape', 'acquisition': {'price_amount': 4200000000}},
 {'name': 'PayPal', 'acquisition': {'price_amount': 1500000000}},
 {'name': 'Zappos', 'acquisition': {'price_amount': 1200000000}},
 {'name': 'Alibaba', 'acquisition': {'price_amount': 1000000000}},
 {'name': 'Postini', 'acquisition': {'price_amount': 625000000}},
 {'name': 'Danger', 'acquisition': {'price_amount': 500000000}},
 {'name': 'Clearwell Systems', 'acquisition': {'price_amount': 410000000}},
 {'name': 'PrimeSense', 'acquisition': {'price_amount': 345000000}},
 {'name': 'Amobee', 'acquisition': {'price_amount': 321000000}},
 {'name': 'Vitrue', 'acquisition': {'price_amount': 300000000}}]

### 16. All the companies on the 'web' `category` that have more than 4000 employees. Sort them by the amount of employees in ascending order.

In [57]:
proj = {"_id": 0, "name": 1, "number_of_employees": 1}
filt = {"$and" : [{"category_code": "web"},{"number_of_employees": {"$gt": 4000}}]}

list(companies.find(filt, proj).sort("number_of_employees", 1).limit(5))

[{'name': 'Expedia', 'number_of_employees': 4400},
 {'name': 'AOL', 'number_of_employees': 8000},
 {'name': 'Webkinz', 'number_of_employees': 8657},
 {'name': 'Los Angeles Times Media Group', 'number_of_employees': 10000},
 {'name': 'Rakuten', 'number_of_employees': 10000}]

### 17. All the companies whose acquisition amount is more than 10.000.000, and currency is 'EUR'.

In [59]:
proj = {"_id": 0, "name":1, "acquisition.price_amount": 1}
filt = {"$and" : [{"acquisition.price_amount":{"$gt":10_000_000}}, {"acquisition.price_currency_code": "EUR"}]}

list(companies.find(filt, proj).limit(5))

[{'name': 'ZYB', 'acquisition': {'price_amount': 31500000}},
 {'name': 'Apertio', 'acquisition': {'price_amount': 140000000}},
 {'name': 'Greenfield Online', 'acquisition': {'price_amount': 40000000}},
 {'name': 'Webedia', 'acquisition': {'price_amount': 70000000}},
 {'name': 'Wayfinder', 'acquisition': {'price_amount': 24000000}}]

### 18. All the companies that have been acquired on the first trimester of the year. Limit the search to 10 companies, and retrieve only their `name` and `acquisition` fields.

In [64]:
proj = {"_id": 0, "name": 1, "acquisition": 1}
filt = {"acquisition.acquired_month": {"$in": [1,2,3]}}

list(companies.find(filt, proj).limit(10))

[{'name': 'Kyte',
  'acquisition': {'price_amount': None,
   'price_currency_code': 'USD',
   'term_code': None,
   'source_url': 'http://techcrunch.com/2011/01/31/exclusive-kit-digital-acquires-kickapps-kewego-and-kyte-for-77-2-million/',
   'source_description': 'KIT digital Acquires KickApps, Kewego AND Kyte For $77.2 Million',
   'acquired_year': 2011,
   'acquired_month': 1,
   'acquired_day': 31,
   'acquiring_company': {'name': 'KIT digital', 'permalink': 'kit-digital'}}},
 {'name': 'NetRatings',
  'acquisition': {'price_amount': 327000000,
   'price_currency_code': 'USD',
   'term_code': 'cash',
   'source_url': 'http://login.vnuemedia.com/hr/login/login_subscribe.jsp?id=0oqDem1gYIfIclz9i2%2Ffqj5NxCp2AC5DPbVnyT2da8GyV2mXjasabE128n69OrmcAh52%2FGE3pSG%2F%0AEKRYD9vh9EhrJrxukmUzh532fSMTZXL42gwPB80UWVtF1NwJ5UZSM%2BCkLU1mpYBoHFgiH%2Fi0f6Ax%0A9yMIVxt47t%2BHamhEQ0nkOEK24L',
   'source_description': 'Nielsen buys rest of NetRatings',
   'acquired_year': 2007,
   'acquired_month': 2,
   

# Bonus
### 19. All the companies that have been founded between 2000 and 2010, but have not been acquired before 2011.

In [66]:
proj = {"_id": 0, "name":1}
filt = {"$and": [{"founded_year": {"$in" :[2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010]}}, {"acquisition.acquired_year": {"$gt": 2011}}]}

list(companies.find(filt, proj).limit(5))

[{'name': 'Wetpaint'},
 {'name': 'Digg'},
 {'name': 'Geni'},
 {'name': 'blogTV'},
 {'name': 'Revision3'}]

### 20. All the companies that have been 'deadpooled' after the third year.

In [ ]:
# Your Code